In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [61]:
# Get the column names

# the official website for Taiwanese Dictionary
url = 'https://sutian.moe.edu.tw/zh-hant/huliok/106/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser') 
table = soup.find_all('table')

# Find all <th> elements (table headers)
titles = soup.find_all('th')  

# Extract text from the <th> elements
table_titles = [title.text.strip() for title in titles]

# Create a DataFrame with the first four titles as headers
headers = table_titles[:4]
df = pd.DataFrame(columns=headers)

# Add an additional 'href' column for audio URL
df['href'] = None

In [62]:
# function to scrap one page
def scrapSinglePage(page):
    pageNum = str(page)
    url = 'https://sutian.moe.edu.tw/zh-hant/huliok/106/?iahbe=' + pageNum # concat the URL and page number
    curPage = requests.get(url)
    soup = BeautifulSoup(curPage.text, 'html')
    table = soup.find_all('table')

    
    for row in table[:-1]:
        # last row of table is not a valid row 
        row_data = row.find_all('td')
        a_tag  = row.find('a')
        href_value = a_tag['href']
        individual_row_data = []
        for data in row_data:
            # Strip and clean the text
            cleaned_text = data.text.strip().replace('\n', '').replace('\t', '').replace('\r', '')
            # Find the first period and keep everything up to and including it
            period_index = cleaned_text.find('.')
            
            if period_index != -1:
                cleaned_text = cleaned_text[:period_index + 1]
            
        
            individual_row_data.append(cleaned_text)

        # The URL for audio
        individual_row_data.append("https://sutian.moe.edu.tw/" + href_value)
        length = len(df)
        df.loc[length] = individual_row_data


In [63]:
# Scrap all 20 pages
for i in range(20):
    scrapSinglePage(i)
df

,序號,詞目,音讀,釋義,href
0,1,鴨仔聽雷。,Ah-á thiann luî.,指鴨子聽到雷聲，並不知道是怎麼回事。比喻一個人對所接收的訊息無法理解。比喻一個人對所接收的訊...,https://sutian.moe.edu.tw//zh-hant/su/27204/
1,2,愛拚才會贏。,Ài piànn tsiah ē iânn.,要努力才會贏。比喻唯有努力奮鬥，才能克服困難，成就一番事業。,https://sutian.moe.edu.tw//zh-hant/su/27161/
2,3,愛媠毋驚流鼻水。,Ài-suí m̄ kiann lâu phīnn-tsuí.,為了展示自己姣好的身材，天氣再冷，寧可挨寒受凍，也不願多穿衣服。常用以揶揄女性愛美，寧可付出...,https://sutian.moe.edu.tw//zh-hant/su/27162/
3,4,暗頓減食一口，活甲九十九。,"Àm-tǹg kiám tsia̍h tsi̍t kháu, ua̍h kah káu-ts...",晚餐少吃一口，活到九十九。說明晚餐的食用量少一點，有益身體健康。,https://sutian.moe.edu.tw//zh-hant/su/27164/
4,5,翁仔某是相欠債。,Ang-á-bóo sī sio-khiàm-tsè.,今世有緣結為夫妻，乃因前世因果，互相欠債所致。夫妻意見不合相互爭吵時，常用這句話來勸雙方姻緣...,https://sutian.moe.edu.tw//zh-hant/su/27093/
...,...,...,...,...,...
375,376,有樣看樣，無樣家己想。,"Ū iūnn khuànn iūnn, bô iūnn ka-kī siūnn.",有樣學樣，沒樣自己想。意為有榜樣可學就依樣畫葫蘆，否則就自己想辦法；或指除了模仿他人，也應自...,https://sutian.moe.edu.tw//zh-hant/su/27009/
376,377,有囝有囝命，無囝天註定。,"Ū kiánn ū kiánn miā, bô kiánn thinn tsù-tiānn.",有孩子是命運的安排，沒孩子是天註定的。用來安慰膝下無子的人不要過於強求。,https://sutian.moe.edu.tw//zh-hant/su/27003/
377,378,有量就有福。,Ū-liōng tō ū hok.,有度量就有福氣。指有度量的人自然會受到上天眷顧，得到賜福。,https://sutian.moe.edu.tw//zh-hant/su/27008/
378,379,有燒香有保庇，有食藥有行氣。,"Ū sio-hiunn ū pó-pì, ū tsia̍h io̍h ū kiânn-khì.",燒香拜佛就會得到神佛保佑，吃藥就會有療效。比喻只要努力或是付出，就會有所收穫或是助益。亦即一...,https://sutian.moe.edu.tw//zh-hant/su/27010/


In [56]:
df_copy = df.copy(deep=True)

In [64]:
# Update header to English
df.columns = ['id', 'phrases', 'pronounciation', 'definition', 'audiourl']
df

,id,phrases,pronounciation,definition,audiourl
0,1,鴨仔聽雷。,Ah-á thiann luî.,指鴨子聽到雷聲，並不知道是怎麼回事。比喻一個人對所接收的訊息無法理解。比喻一個人對所接收的訊...,https://sutian.moe.edu.tw//zh-hant/su/27204/
1,2,愛拚才會贏。,Ài piànn tsiah ē iânn.,要努力才會贏。比喻唯有努力奮鬥，才能克服困難，成就一番事業。,https://sutian.moe.edu.tw//zh-hant/su/27161/
2,3,愛媠毋驚流鼻水。,Ài-suí m̄ kiann lâu phīnn-tsuí.,為了展示自己姣好的身材，天氣再冷，寧可挨寒受凍，也不願多穿衣服。常用以揶揄女性愛美，寧可付出...,https://sutian.moe.edu.tw//zh-hant/su/27162/
3,4,暗頓減食一口，活甲九十九。,"Àm-tǹg kiám tsia̍h tsi̍t kháu, ua̍h kah káu-ts...",晚餐少吃一口，活到九十九。說明晚餐的食用量少一點，有益身體健康。,https://sutian.moe.edu.tw//zh-hant/su/27164/
4,5,翁仔某是相欠債。,Ang-á-bóo sī sio-khiàm-tsè.,今世有緣結為夫妻，乃因前世因果，互相欠債所致。夫妻意見不合相互爭吵時，常用這句話來勸雙方姻緣...,https://sutian.moe.edu.tw//zh-hant/su/27093/
...,...,...,...,...,...
375,376,有樣看樣，無樣家己想。,"Ū iūnn khuànn iūnn, bô iūnn ka-kī siūnn.",有樣學樣，沒樣自己想。意為有榜樣可學就依樣畫葫蘆，否則就自己想辦法；或指除了模仿他人，也應自...,https://sutian.moe.edu.tw//zh-hant/su/27009/
376,377,有囝有囝命，無囝天註定。,"Ū kiánn ū kiánn miā, bô kiánn thinn tsù-tiānn.",有孩子是命運的安排，沒孩子是天註定的。用來安慰膝下無子的人不要過於強求。,https://sutian.moe.edu.tw//zh-hant/su/27003/
377,378,有量就有福。,Ū-liōng tō ū hok.,有度量就有福氣。指有度量的人自然會受到上天眷顧，得到賜福。,https://sutian.moe.edu.tw//zh-hant/su/27008/
378,379,有燒香有保庇，有食藥有行氣。,"Ū sio-hiunn ū pó-pì, ū tsia̍h io̍h ū kiânn-khì.",燒香拜佛就會得到神佛保佑，吃藥就會有療效。比喻只要努力或是付出，就會有所收穫或是助益。亦即一...,https://sutian.moe.edu.tw//zh-hant/su/27010/


In [65]:
# Add columns 
df.insert(4, 'tags', "")
df.insert(4, 'usage', "Proverb")
df

,id,phrases,pronounciation,definition,usage,tags,audiourl
0,1,鴨仔聽雷。,Ah-á thiann luî.,指鴨子聽到雷聲，並不知道是怎麼回事。比喻一個人對所接收的訊息無法理解。比喻一個人對所接收的訊...,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27204/
1,2,愛拚才會贏。,Ài piànn tsiah ē iânn.,要努力才會贏。比喻唯有努力奮鬥，才能克服困難，成就一番事業。,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27161/
2,3,愛媠毋驚流鼻水。,Ài-suí m̄ kiann lâu phīnn-tsuí.,為了展示自己姣好的身材，天氣再冷，寧可挨寒受凍，也不願多穿衣服。常用以揶揄女性愛美，寧可付出...,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27162/
3,4,暗頓減食一口，活甲九十九。,"Àm-tǹg kiám tsia̍h tsi̍t kháu, ua̍h kah káu-ts...",晚餐少吃一口，活到九十九。說明晚餐的食用量少一點，有益身體健康。,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27164/
4,5,翁仔某是相欠債。,Ang-á-bóo sī sio-khiàm-tsè.,今世有緣結為夫妻，乃因前世因果，互相欠債所致。夫妻意見不合相互爭吵時，常用這句話來勸雙方姻緣...,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27093/
...,...,...,...,...,...,...,...
375,376,有樣看樣，無樣家己想。,"Ū iūnn khuànn iūnn, bô iūnn ka-kī siūnn.",有樣學樣，沒樣自己想。意為有榜樣可學就依樣畫葫蘆，否則就自己想辦法；或指除了模仿他人，也應自...,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27009/
376,377,有囝有囝命，無囝天註定。,"Ū kiánn ū kiánn miā, bô kiánn thinn tsù-tiānn.",有孩子是命運的安排，沒孩子是天註定的。用來安慰膝下無子的人不要過於強求。,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27003/
377,378,有量就有福。,Ū-liōng tō ū hok.,有度量就有福氣。指有度量的人自然會受到上天眷顧，得到賜福。,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27008/
378,379,有燒香有保庇，有食藥有行氣。,"Ū sio-hiunn ū pó-pì, ū tsia̍h io̍h ū kiânn-khì.",燒香拜佛就會得到神佛保佑，吃藥就會有療效。比喻只要努力或是付出，就會有所收穫或是助益。亦即一...,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27010/


In [66]:
# Loop through the data frame, call API for translation of definition
for i in range(len(df)):
    chinese = df.at[i, 'definition']
    
    response = requests.post(
        "http://localhost:8000/translate/invoke",
        json = {
            'input': {'Definition' : chinese}}
    )
    translated_def = response.json()['output']['content']
    df.at[i, 'definition'] = translated_def
df

,id,phrases,pronounciation,definition,usage,tags,audiourl
0,1,鴨仔聽雷。,Ah-á thiann luî.,The text you provided is already translated in...,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27204/
1,2,愛拚才會贏。,Ài piànn tsiah ē iânn.,"""Effort leads to victory. A metaphor that only...",Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27161/
2,3,愛媠毋驚流鼻水。,Ài-suí m̄ kiann lâu phīnn-tsuí.,"""To show off her attractive figure, no matter ...",Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27162/
3,4,暗頓減食一口，活甲九十九。,"Àm-tǹg kiám tsia̍h tsi̍t kháu, ua̍h kah káu-ts...","""Eat one less bite at dinner, live until ninet...",Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27164/
4,5,翁仔某是相欠債。,Ang-á-bóo sī sio-khiàm-tsè.,"""In this life, becoming husband and wife is du...",Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27093/
...,...,...,...,...,...,...,...
375,376,有樣看樣，無樣家己想。,"Ū iūnn khuànn iūnn, bô iūnn ka-kī siūnn.","""Monkey see, monkey do; no model, think for yo...",Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27009/
376,377,有囝有囝命，無囝天註定。,"Ū kiánn ū kiánn miā, bô kiánn thinn tsù-tiānn.","""Having children is a matter of fate, not havi...",Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27003/
377,378,有量就有福。,Ū-liōng tō ū hok.,"""Having magnanimity brings blessings. It means...",Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27008/
378,379,有燒香有保庇，有食藥有行氣。,"Ū sio-hiunn ū pó-pì, ū tsia̍h io̍h ū kiânn-khì.",Burning incense and worshiping Buddha will bri...,Proverb,,https://sutian.moe.edu.tw//zh-hant/su/27010/


In [70]:
# Loop through the data frame, call API for adding tags
for i in range(len(df)):
    phrase = df.at[i, 'usage'] + ": " + df.at[i, 'phrase']
    # print(phrase)
    
    response = requests.post(
        "http://localhost:8000/categorize/invoke",
        json = {
            'input': {'phrase' : phrase}}
    )
    tags = response.json()['output']['content']
    # print(tags)
    df.at[i, 'tags'] = tags
df

,id,phrase,pronounciation,definition,usage,tags,audiourl
0,1,鴨仔聽雷。,Ah-á thiann luî.,The text you provided is already translated in...,Proverb,"funny,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27204/
1,2,愛拚才會贏。,Ài piànn tsiah ē iânn.,"""Effort leads to victory. A metaphor that only...",Proverb,"perseverance,success",https://sutian.moe.edu.tw//zh-hant/su/27161/
2,3,愛媠毋驚流鼻水。,Ài-suí m̄ kiann lâu phīnn-tsuí.,"""To show off her attractive figure, no matter ...",Proverb,"funny,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27162/
3,4,暗頓減食一口，活甲九十九。,"Àm-tǹg kiám tsia̍h tsi̍t kháu, ua̍h kah káu-ts...","""Eat one less bite at dinner, live until ninet...",Proverb,wisdom,https://sutian.moe.edu.tw//zh-hant/su/27164/
4,5,翁仔某是相欠債。,Ang-á-bóo sī sio-khiàm-tsè.,"""In this life, becoming husband and wife is du...",Proverb,"relationships,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27093/
...,...,...,...,...,...,...,...
375,376,有樣看樣，無樣家己想。,"Ū iūnn khuànn iūnn, bô iūnn ka-kī siūnn.","""Monkey see, monkey do; no model, think for yo...",Proverb,wisdom,https://sutian.moe.edu.tw//zh-hant/su/27009/
376,377,有囝有囝命，無囝天註定。,"Ū kiánn ū kiánn miā, bô kiánn thinn tsù-tiānn.","""Having children is a matter of fate, not havi...",Proverb,"relationships,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27003/
377,378,有量就有福。,Ū-liōng tō ū hok.,"""Having magnanimity brings blessings. It means...",Proverb,"wisdom,success",https://sutian.moe.edu.tw//zh-hant/su/27008/
378,379,有燒香有保庇，有食藥有行氣。,"Ū sio-hiunn ū pó-pì, ū tsia̍h io̍h ū kiânn-khì.",Burning incense and worshiping Buddha will bri...,Proverb,"wisdom,hardship",https://sutian.moe.edu.tw//zh-hant/su/27010/


In [71]:
# Add a mandarin column
df.insert(3, "mandarin", "")
df

,id,phrase,pronounciation,mandarin,definition,usage,tags,audiourl
0,1,鴨仔聽雷。,Ah-á thiann luî.,,The text you provided is already translated in...,Proverb,"funny,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27204/
1,2,愛拚才會贏。,Ài piànn tsiah ē iânn.,,"""Effort leads to victory. A metaphor that only...",Proverb,"perseverance,success",https://sutian.moe.edu.tw//zh-hant/su/27161/
2,3,愛媠毋驚流鼻水。,Ài-suí m̄ kiann lâu phīnn-tsuí.,,"""To show off her attractive figure, no matter ...",Proverb,"funny,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27162/
3,4,暗頓減食一口，活甲九十九。,"Àm-tǹg kiám tsia̍h tsi̍t kháu, ua̍h kah káu-ts...",,"""Eat one less bite at dinner, live until ninet...",Proverb,wisdom,https://sutian.moe.edu.tw//zh-hant/su/27164/
4,5,翁仔某是相欠債。,Ang-á-bóo sī sio-khiàm-tsè.,,"""In this life, becoming husband and wife is du...",Proverb,"relationships,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27093/
...,...,...,...,...,...,...,...,...
375,376,有樣看樣，無樣家己想。,"Ū iūnn khuànn iūnn, bô iūnn ka-kī siūnn.",,"""Monkey see, monkey do; no model, think for yo...",Proverb,wisdom,https://sutian.moe.edu.tw//zh-hant/su/27009/
376,377,有囝有囝命，無囝天註定。,"Ū kiánn ū kiánn miā, bô kiánn thinn tsù-tiānn.",,"""Having children is a matter of fate, not havi...",Proverb,"relationships,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27003/
377,378,有量就有福。,Ū-liōng tō ū hok.,,"""Having magnanimity brings blessings. It means...",Proverb,"wisdom,success",https://sutian.moe.edu.tw//zh-hant/su/27008/
378,379,有燒香有保庇，有食藥有行氣。,"Ū sio-hiunn ū pó-pì, ū tsia̍h io̍h ū kiânn-khì.",,Burning incense and worshiping Buddha will bri...,Proverb,"wisdom,hardship",https://sutian.moe.edu.tw//zh-hant/su/27010/


In [72]:
# Add mandarin translation 
for i in range(len(df)):
    phrase = df.at[i, 'phrase']
    definition = df.at[i, 'definition']
    response = requests.post(
        "http://localhost:8000/translate-to-mandarin/invoke",
        json = {
            'input': {'phrase' : phrase, 'definition': definition}}
    )
    df.at[i, 'mandarin'] = response.json()['output']['content']

df

,id,phrase,pronounciation,mandarin,definition,usage,tags,audiourl
0,1,鴨仔聽雷。,Ah-á thiann luî.,鴨子聽到雷,The text you provided is already translated in...,Proverb,"funny,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27204/
1,2,愛拚才會贏。,Ài piànn tsiah ē iânn.,愛拼才會贏,"""Effort leads to victory. A metaphor that only...",Proverb,"perseverance,success",https://sutian.moe.edu.tw//zh-hant/su/27161/
2,3,愛媠毋驚流鼻水。,Ài-suí m̄ kiann lâu phīnn-tsuí.,愛美不怕冷，寧願凍著也不穿多。,"""To show off her attractive figure, no matter ...",Proverb,"funny,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27162/
3,4,暗頓減食一口，活甲九十九。,"Àm-tǹg kiám tsia̍h tsi̍t kháu, ua̍h kah káu-ts...",晚餐少吃一口，活到九十九。,"""Eat one less bite at dinner, live until ninet...",Proverb,wisdom,https://sutian.moe.edu.tw//zh-hant/su/27164/
4,5,翁仔某是相欠債。,Ang-á-bóo sī sio-khiàm-tsè.,夫妻是前世的債欠,"""In this life, becoming husband and wife is du...",Proverb,"relationships,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27093/
...,...,...,...,...,...,...,...,...
375,376,有樣看樣，無樣家己想。,"Ū iūnn khuànn iūnn, bô iūnn ka-kī siūnn.",有樣看樣，無樣自己想。,"""Monkey see, monkey do; no model, think for yo...",Proverb,wisdom,https://sutian.moe.edu.tw//zh-hant/su/27009/
376,377,有囝有囝命，無囝天註定。,"Ū kiánn ū kiánn miā, bô kiánn thinn tsù-tiānn.",有孩子是命中注定，沒有孩子是天意。,"""Having children is a matter of fate, not havi...",Proverb,"relationships,wisdom",https://sutian.moe.edu.tw//zh-hant/su/27003/
377,378,有量就有福。,Ū-liōng tō ū hok.,有量就有福。,"""Having magnanimity brings blessings. It means...",Proverb,"wisdom,success",https://sutian.moe.edu.tw//zh-hant/su/27008/
378,379,有燒香有保庇，有食藥有行氣。,"Ū sio-hiunn ū pó-pì, ū tsia̍h io̍h ū kiânn-khì.",有燒香有保庇，有服藥有行氣。,Burning incense and worshiping Buddha will bri...,Proverb,"wisdom,hardship",https://sutian.moe.edu.tw//zh-hant/su/27010/


In [73]:
# format the tag to import to psql column text[]
for i in range(len(df)):
    tags_string = df.at[i, "tags"]
    tags_array = tags_string.split(",")
    new_tags_string = "\"{"
    for j in range(len(tags_array)):
        tags_array[j] = tags_array[j].lstrip(' ')
        new_tags_string += tags_array[j] + ","
    tags_string = new_tags_string[:-1] + "}\""
    df.at[i, "tags"] = tags_string

df

,id,phrase,pronounciation,mandarin,definition,usage,tags,audiourl
0,1,鴨仔聽雷。,Ah-á thiann luî.,鴨子聽到雷,The text you provided is already translated in...,Proverb,"""{funny,wisdom}""",https://sutian.moe.edu.tw//zh-hant/su/27204/
1,2,愛拚才會贏。,Ài piànn tsiah ē iânn.,愛拼才會贏,"""Effort leads to victory. A metaphor that only...",Proverb,"""{perseverance,success}""",https://sutian.moe.edu.tw//zh-hant/su/27161/
2,3,愛媠毋驚流鼻水。,Ài-suí m̄ kiann lâu phīnn-tsuí.,愛美不怕冷，寧願凍著也不穿多。,"""To show off her attractive figure, no matter ...",Proverb,"""{funny,wisdom}""",https://sutian.moe.edu.tw//zh-hant/su/27162/
3,4,暗頓減食一口，活甲九十九。,"Àm-tǹg kiám tsia̍h tsi̍t kháu, ua̍h kah káu-ts...",晚餐少吃一口，活到九十九。,"""Eat one less bite at dinner, live until ninet...",Proverb,"""{wisdom}""",https://sutian.moe.edu.tw//zh-hant/su/27164/
4,5,翁仔某是相欠債。,Ang-á-bóo sī sio-khiàm-tsè.,夫妻是前世的債欠,"""In this life, becoming husband and wife is du...",Proverb,"""{relationships,wisdom}""",https://sutian.moe.edu.tw//zh-hant/su/27093/
...,...,...,...,...,...,...,...,...
375,376,有樣看樣，無樣家己想。,"Ū iūnn khuànn iūnn, bô iūnn ka-kī siūnn.",有樣看樣，無樣自己想。,"""Monkey see, monkey do; no model, think for yo...",Proverb,"""{wisdom}""",https://sutian.moe.edu.tw//zh-hant/su/27009/
376,377,有囝有囝命，無囝天註定。,"Ū kiánn ū kiánn miā, bô kiánn thinn tsù-tiānn.",有孩子是命中注定，沒有孩子是天意。,"""Having children is a matter of fate, not havi...",Proverb,"""{relationships,wisdom}""",https://sutian.moe.edu.tw//zh-hant/su/27003/
377,378,有量就有福。,Ū-liōng tō ū hok.,有量就有福。,"""Having magnanimity brings blessings. It means...",Proverb,"""{wisdom,success}""",https://sutian.moe.edu.tw//zh-hant/su/27008/
378,379,有燒香有保庇，有食藥有行氣。,"Ū sio-hiunn ū pó-pì, ū tsia̍h io̍h ū kiânn-khì.",有燒香有保庇，有服藥有行氣。,Burning incense and worshiping Buddha will bri...,Proverb,"""{wisdom,hardship}""",https://sutian.moe.edu.tw//zh-hant/su/27010/


In [ ]:
df.to_csv('/Users/yushanlu/Desktop/yushan-says/backend/scripts/data.csv', sep='|', index=False)